In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salary-dataset-simple-linear-regression/Salary_dataset.csv


In [125]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [128]:
df=pd.read_csv('/kaggle/input/salary-dataset-simple-linear-regression/Salary_dataset.csv')

In [6]:
# df.drop('Unnamed: 0',axis=1,inplace=True)

#### Functions to info the data set

In [176]:
def  data_info(data):
    cols=[]
    unique_val=[]
    n_uniques=[]
    dtypes=[]
    nulss=[]
    dup=[]
    is_cat=[]
    for col in data.columns:
        cols.append(col)
        dtypes.append(data[col].dtype)
        unique_val.append(data[col].value_counts().head(10).index.unique().values)
        n_uniques.append(data[col].nunique())
        nulss.append(data[col].isnull().sum())
        dup.append(data.duplicated().sum())
    return pd.DataFrame({'Col':cols,'dtype':dtypes,'n_uniques':n_uniques,'n_nan':nulss,'unique_val':unique_val,'duplicated':dup})

In [177]:
data_info(df)

,Col,dtype,n_uniques,n_nan,unique_val,duplicated
0,Unnamed: 0,int64,30,0,"[0, 1, 28, 27, 26, 25, 24, 23, 22, 21]",0
1,YearsExperience,float64,28,0,"[3.3000000000000003, 4.1, 1.2000000000000002, ...",0
2,Salary,float64,30,0,"[39344.0, 46206.0, 122392.0, 112636.0, 116970....",0


#### Functions to drop outlier

In [140]:
def clean_data(data,columns):
    for col in columns:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower_whisker = q1 - 1.5 * iqr
        upper_whisker = q3 + 1.5 * iqr
        filt_lower = data[col] < lower_whisker
        filt_upper = data[col] > upper_whisker
        filt = filt_lower | filt_upper
        outlier_indices = data[filt].index
        data.drop(outlier_indices, axis = 0, inplace = True)
    return data

In [141]:
clean_data(df,columns=df.columns)

,Unnamed: 0,YearsExperience,Salary
0,0,1.2,39344.0
1,1,1.4,46206.0
2,2,1.6,37732.0
3,3,2.1,43526.0
4,4,2.3,39892.0
5,5,3.0,56643.0
6,6,3.1,60151.0
7,7,3.3,54446.0
8,8,3.3,64446.0
9,9,3.8,57190.0


In [201]:
x=df.drop('YearsExperience',axis=1)
y=df['YearsExperience']

In [202]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [243]:
dict_classifiers = [LinearRegression(), KNeighborsRegressor()]

In [270]:
from sklearn.linear_model import LinearRegression 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_squared_error,accuracy_score
per=[]
def displaymetrics( models: list, X_train: np.array,y_train: np.array, X_test: np.array,y_test: np.array,Reg):
    for model in models:  
        model.fit(X_train, y_train)
        y_score =model.predict(X_test)
        if bool(Reg)==True:
            mse=mean_squared_error(y_score,y_test)
            per.append(mse)
        else:
            acu=accuracy_score(y_score,y_test)
            
    return pd.DataFrame({"Models":models,'Mse_or_acc':sorted(per)})

In [271]:
displaymetrics(dict_classifiers,X_train,y_train,X_test,y_test,Reg=True)

,Models,Mse_or_acc
0,LinearRegression(),0.140069
1,KNeighborsRegressor(),0.675400
